In [1]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [2]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# # filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

# _filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results
filename_pickled_df_a = 'df_OHLCV_clean'  # df adjusted OHLCV, dropped symbols with no vol and close
filename_pickled_df_c = 'df_close_clean'  # df close, dropped symbols with no vol and close

verbose = False  # True prints more output

#################
# look_back_days = -250 * 60  # subset df iloc days
look_back_days = -250 * 6  # subset df iloc days, 6 years of data
#################

In [3]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [4]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_adjOHLCV[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['AMCR', 'AVT', 'BIIB', 'BKR', 'CCI', 'CHD', 'CNC', 'CNXC', 'ESAB',
       'FCFS', 'GEN', 'IBKR', 'ICPT', 'JJC', 'PEAK', 'PFG', 'SBNY', 'SGEN',
       'SRPT', 'TWNK', 'VC', 'VMW', 'VNT', 'WH', 'XEL'],
      dtype='object')


In [5]:
df_dif = df_v - df_v.shift(periods=1)
rows, cols = np.where(df_dif == 0)
idx_same_volume = list(set(cols))
idx_same_volume.sort()
idx_same_volume
symbols_same_volume = df_v.columns[idx_same_volume]
print(f'symbols with same volume:\n{symbols_same_volume}')

symbols with same volume:
Index(['ABM', 'ACM', 'ACN', 'ADP', 'AGCO', 'ALRM', 'AMCR', 'AMED', 'AMG',
       'AMGN',
       ...
       'VRE', 'VRSK', 'VSAT', 'WERN', 'WEX', 'WSFS', 'WTS', 'WTW', 'XEL',
       'XENE'],
      dtype='object', length=246)


In [6]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['ABNB', 'AIRC', 'ALGM', 'BJ', 'BRBR', 'BTC-USD', 'CARR', 'CDAY', 'CEG',
       'CHK', 'CHX', 'CNXC', 'CR', 'CRWD', 'CTVA', 'DDOG', 'DOW', 'DT', 'DTM',
       'ESAB', 'ETH-USD', 'ETRN', 'FOX', 'FOXA', 'FYBR', 'GEHC', 'GFS', 'GO',
       'GXO', 'HEP', 'ICPT', 'JJC', 'KD', 'LCID', 'MP', 'MRNA', 'NARI', 'NVST',
       'NVT', 'OGN', 'OSTK', 'OTIS', 'PACW', 'PDD', 'PGNY', 'RIVN', 'SGEN',
       'SGOV', 'SHC', 'SITM', 'SWAV', 'TWNK', 'VMW', 'VNT', 'VSCO', 'WH',
       'YETI', 'ZM', 'ZS'],
      dtype='object')


In [7]:
symbols_bad_data = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
unique_symbols_bad_data = sorted(list(set(symbols_bad_data)))  # unique symbols
print(f'unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: {len(unique_symbols_bad_data)}')

unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: 305


In [8]:
# get symbols of past model picks
df_picks = pickle_load(path_data_dump, 'df_picks', verbose=verbose)
df_picks.head(1)

,date_end_df_train,max_days_lookbacks,days_lookbacks,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,2023-12-19,120,"[30, 60, 120]",[],[],[],[],[],[],[],"['GPS', 'SHV']",['FTSM'],[],"['ANF', 'HIBB', 'SQ', 'X']","['AMKR', 'LRN', 'MARA', 'MBI', 'VRNS']","['BA', 'BPMC', 'BTC-USD', 'EDU', 'FFWM', 'GBTC...",[]


In [9]:
# keep only columns with symbols of past picks
df_picks.drop(columns=["date_end_df_train", "max_days_lookbacks", "days_lookbacks"], inplace=True)
df_picks.head(1)

,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,[],[],[],[],[],[],[],"['GPS', 'SHV']",['FTSM'],[],"['ANF', 'HIBB', 'SQ', 'X']","['AMKR', 'LRN', 'MARA', 'MBI', 'VRNS']","['BA', 'BPMC', 'BTC-USD', 'EDU', 'FFWM', 'GBTC...",[]


In [10]:
df_picks.columns

Index(['sym_freq_15', 'sym_freq_14', 'sym_freq_13', 'sym_freq_12',
       'sym_freq_11', 'sym_freq_10', 'sym_freq_9', 'sym_freq_8', 'sym_freq_7',
       'sym_freq_6', 'sym_freq_5', 'sym_freq_4', 'sym_freq_3', 'sym_freq_2'],
      dtype='object')

In [11]:
import ast

In [12]:
list_of_lists = []

for col in df_picks.columns:
  # convert column values from string to list, e.g. '[]', '[]', '["A", "B", ..]' ... to [], [], ["A", "B", ..], ...
  l_series = df_picks[col].apply(ast.literal_eval)
  # list_of_lists = [l_item for l_item in l_series if l_item]  # this doesn't works  
  for l_item in l_series:
      if l_item:  # 
        list_of_lists.append(l_item)  

symbols_picks = [val for sublist in list_of_lists for val in sublist]
print(f'symbol count from model picks: {len(symbols_picks)}')
# list sorted unique symbols
unique_symbols_picks = sorted(list(set(symbols_picks)))
print(f'unique symbol count from model picks: {len(unique_symbols_picks)}')

symbol count from model picks: 7760
unique symbol count from model picks: 337


In [13]:
def find_common_items(list1, list2):
  """Finds the common items between two lists.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the common items between the two lists.
  """

  common_items = []
  for item in list1:
    if item in list2:
      common_items.append(item)
  return common_items


# code in if block runs only from command line, code will NOT be executed if imported as a module
if __name__ == "__main__": 
  list1 = ["a", "b", "c", "d", "e"]
  list2 = ["b", "c", "d", "f", "g"]
  common_items = find_common_items(list1, list2)
  print(common_items)

['b', 'c', 'd']


In [14]:
# symbols are in past picks but also have bad data 
common_symbols = find_common_items(unique_symbols_picks, unique_symbols_bad_data)
print(common_symbols)

['BTC-USD', 'CHT', 'CMG', 'CVBF', 'CVLT', 'DVA', 'ETH-USD', 'HEP', 'HOLI', 'HY', 'ICPT', 'MLR', 'OSIS', 'OSTK', 'PKX', 'SFBS', 'SGEN', 'SRPT', 'TWNK', 'VNO', 'XENE']


In [15]:
def subtract_items(list1, list2):
  """Subtracts items in list2 from items in list1.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the items in list1 that are not in list2.
  """

  subtracted_items = []
  for item in list1:
    if item not in list2:
      subtracted_items.append(item)
  return subtracted_items

In [16]:
symbols_drop = subtract_items(unique_symbols_bad_data, common_symbols)  # don't drop symbols in past picks
symbols_drop .sort()
print(f'len(unique_symbols_bad_data): {len(unique_symbols_bad_data)}')
print(f'len(common_symbols): {len(common_symbols)}')
print(f'len(symbols_drop): {len(symbols_drop)}')

len(unique_symbols_bad_data): 305
len(common_symbols): 21
len(symbols_drop): 284


In [17]:
# symbols_drop = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
# print(f'combined symbols with no volume, same volume and no close, inculdes duplicate symbols: {len(symbols_drop)}')
# symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
# symbols_drop .sort()


df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
print(f'unique symbols dropped from df_a (adjOLHLV) and df_c (Close): {len(symbols_drop)}')

unique symbols dropped from df_a (adjOLHLV) and df_c (Close): 284


In [18]:
print(f'symbols with no volume:   {len(symbols_no_volume):>5,}')
print(f'symbols with same volume: {len(symbols_same_volume):>5,}')
print(f'symbols with no close:    {len(symbols_no_close):>5,}\n')
print(f'symbols total before drop:                                        {len(df_close.columns):>5,}')
print(f'unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): {len(symbols_drop):>5,}\n')
print('                                          symbols     rows')
print(f'df adjOHLCV (df_a) after dropped symbols:   {len(df_a.columns)/5:>5,.0f}    {len(df_a):>5,}')
print(f'df Close (df_c) after dropped symbols:      {len(df_c.columns):>5,}    {len(df_c):>5,}')


symbols with no volume:      25
symbols with same volume:   246
symbols with no close:       59

symbols total before drop:                                        1,554
unique symbols dropped from df OHLCVA (df_a) and df Close (df_c):   284

                                          symbols     rows
df adjOHLCV (df_a) after dropped symbols:   1,270    1,500
df Close (df_c) after dropped symbols:      1,270    1,500


In [19]:
pickle_dump(df_a, path_data_dump, filename_pickled_df_a)
print(f'pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_a}')
pickle_dump(df_c, path_data_dump, filename_pickled_df_c)
print(f'pickled df Close after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_c}')

pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCV_clean
pickled df Close after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_close_clean


In [20]:
from myUtils import list_dump

f_symbols_df_close_clean = 'symbols_df_close_clean.csv'  # symbols text file
symbols_df_c = list(df_c)  # column names in df_c
list_dump(symbols_df_c, path_data_dump, f_symbols_df_close_clean)# df_c.columns.to_csv(f_symbols_df_close_clean)

In [21]:
df_a.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2023-12-14  135.000000  138.169998  135.000000  137.960007  2950500.0   
2023-12-15  137.240005  138.899994  136.279999  136.779999  4876000.0   
2023-12-18  137.589996  138.190002  136.860001  137.529999  1634700.0   
2023-12-19  138.509995  139.889999  137.320007  139.789993  1523300.0   
2023-12-20  139.789993  141.240005  138.139999  138.179993  2120885.0   

                   AA                                               ...  \
                 Open       High        Low      Close      Volume  ...   
Date                                                                ...   
2023-12-14  27.040001  30.135000  26.940001  29.900000  16594000.0  ...   
2023-12-15  29.930000  32.500000  29.670000  31.100000  19183200.0  ...   
2023-12-18  31.440001  32.310001  30.520000  30.520000   7270400.0  ...   
2023-12-19  31.190001  33.000000  30.910000  32.419998   7967200.0  ...   
2023-12-20  32.009998  32.474998  30.559999  30.650000   7405858.0  ...   

                 ZUMZ                                                   ZWS  \
                 Open       High        Low      Close    Volume       Open   
Date                                                                          
2023-12-14  20.059999  20.620001  19.620001  19.930000  276600.0  29.910000   
2023-12-15  20.139999  20.230000  19.209999  19.389999  371600.0  29.660000   
2023-12-18  19.420000  19.500000  19.049999  19.059999  204100.0  29.370001   
2023-12-19  19.180000  19.559999  19.010000  19.459999  220800.0  29.230000   
2023-12-20  19.340000  20.129999  18.840000  18.850000  178542.0  29.879999   

                                                        
                 High        Low      Close     Volume  
Date                                                    
2023-12-14  30.040001  29.160000  29.610001  2380100.0  
2023-12-15  29.959999  29.160000  29.350000  2160300.0  
2023-12-18  29.700001  29.070000  29.629999  1210000.0  
2023-12-19  29.889999  29.049999  29.770000  1260900.0  
2023-12-20  30.370001  29.590000  29.629999  1181302.0  

[5 rows x 6350 columns]

In [22]:
df_c.tail()

,A,AA,AAL,AAP,AAPL,AB,ABBV,ABR,ABT,ACGL,...,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2023-12-14,137.960007,29.900000,14.59,63.970001,198.110001,31.870001,154.880005,15.70,108.599998,74.669998,...,38.799999,118.529999,272.160004,66.320000,54.500000,45.669998,20.309999,200.089996,19.930000,29.610001
2023-12-15,136.779999,31.100000,14.49,62.720001,197.570007,32.020000,154.039993,15.38,107.290001,73.419998,...,38.540001,118.690002,270.179993,66.550003,54.779999,43.840000,19.830000,196.289993,19.389999,29.350000
2023-12-18,137.529999,30.520000,14.24,60.660000,195.889999,32.209999,153.419998,15.36,108.059998,74.650002,...,38.799999,118.800003,266.679993,66.260002,55.330002,43.040001,20.200001,196.720001,19.059999,29.629999
2023-12-19,139.789993,32.419998,14.30,60.860001,196.940002,32.750000,153.600006,16.10,108.419998,74.830002,...,39.880001,119.970001,269.420013,67.510002,56.930000,43.930000,19.940001,198.080002,19.459999,29.770000
2023-12-20,138.179993,30.650000,13.98,59.450001,194.830002,31.620001,151.690002,15.49,107.400002,73.610001,...,39.840000,120.000000,261.609985,66.190002,55.480000,42.540001,19.920000,194.630005,18.850000,29.629999
